## Setup
To begin, you will need to provide the following information about your Azure Subscription.

**If you are using your own Azure subscription, please provide names for subscription_id, resource_group, workspace_name and workspace_region to use.** Note that the workspace needs to be of type [Machine Learning Workspace](https://docs.microsoft.com/en-us/azure/machine-learning/service/setup-create-workspace).

**If an environment is provided to you be sure to replace XXXXX in the values below with your unique identifier.**

In the following cell, be sure to set the values for `subscription_id`, `resource_group`, `workspace_name` and `workspace_region` as directed by the comments (*these values can be acquired from the Azure Portal*).

To get these values, do the following:
1. Navigate to the Azure Portal and login with the credentials provided.
2. From the left hand menu, under Favorites, select `Resource Groups`.
3. In the list, select the resource group with the name similar to `XXXXX`.
4. From the Overview tab, capture the desired values.

Execute the following cell by selecting the `>|Run` button in the command bar above.

In [1]:
#Provide the Subscription ID of your existing Azure subscription
subscription_id = "" # <- needs to be the subscription with the Quick-Starts resource group

#Provide values for the existing Resource Group 
resource_group = "Quick-Starts-XXXXX" # <- replace XXXXX with your unique identifier

#Provide the Workspace Name and Azure Region of the Azure Machine Learning Workspace
workspace_name = "quick-starts-ws-XXXXX" # <- replace XXXXX with your unique identifier
workspace_region = "eastus" # <- region of your Quick-Starts resource group

experiment_name = 'deep-learning'

### Create and connect to an Azure Machine Learning Workspace

Run the following cell to create a new Azure Machine Learning **Workspace**.

**Important Note**: You will be prompted to login in the text that is output below the cell. Be sure to navigate to the URL displayed and enter the code that is provided. Once you have entered the code, return to this notebook and wait for the output to read `Workspace configuration succeeded`.

In [3]:
from azureml.core import Workspace, Experiment, Run

ws = Workspace.create(
    name = workspace_name,
    subscription_id = subscription_id,
    resource_group = resource_group, 
    location = workspace_region,
    exist_ok = True)

ws.write_config()
print('Workspace configuration succeeded')

Workspace configuration succeeded


## Get the Model Training Run

**Load the run_info.json file that has the run id for the model training run**

In [4]:
import os
import json

output_path = './outputs'

run_info_filepath = os.path.join(output_path, 'run_info.json')

try:
    with open(run_info_filepath) as f:
        run_info = json.load(f)
        print('run_info.json loaded')
        print(run_info)
except:
    print("Cannot open: ", run_info_filepath)
    print("Please fix output_path before proceeding!")

run_info.json loaded
{'id': 'deep-learning_1560901434_60a4db36'}


**Get the Run object from the run id**

In [5]:
run = Run(Experiment(ws, experiment_name), run_info['id'])

## Register Model

Download the model file from the **Run** object

In [6]:
# create a model folder in the current directory
os.makedirs('./model', exist_ok=True)

for f in run.get_file_names():
    if f.startswith('outputs/model'):
        output_file_path = os.path.join('./model', f.split('/')[-1])
        print('Downloading from {} to {} ...'.format(f, output_file_path))
        run.download_file(name=f, output_file_path=output_file_path)

**Register the Model with Azure Model Registry**

In [7]:
from azureml.core.model import Model

model_name = 'compliance-classifier'
model_description = 'Deep learning model to classify the descriptions of car components as compliant or non-compliant.'
model = Model.register(
    model_path=output_file_path,  # this points to a local file
    model_name=model_name,  # this is the name the model is registered as
    tags={"type": "classification", "run_id": run.id},
    description=model_description,
    workspace=ws
)

print("Model registered: {} \nModel Description: {} \nModel Version: {}".format(model.name, model.description, model.version))

Registering model compliance-classifier
Model registered: compliance-classifier 
Model Description: Deep learning model to classify the descriptions of car components as compliant or non-compliant. 
Model Version: 1
